In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [35]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(
        "chromedriver_win32\chromedriver.exe",
        #options=options
        )

C:\Users\inaci\AppData\Local\Temp\ipykernel_39656\1423187972.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [36]:
driver.get('https://www.flashscore.com.br/')

In [37]:
sport = driver.find_element(By.CLASS_NAME, 'sportName')
events = sport.find_elements(By.CLASS_NAME, 'event__match')
len(events)

84

In [38]:
IDS = list()
for event in events:
    ID = event.get_attribute('id')
    ID = ID.replace('g_1_', '')
    IDS.append(ID)

In [39]:
with open("data/diario/02_12/jogos.json","w", encoding='utf-8') as jsonfile:
    json.dump(IDS, jsonfile, ensure_ascii=False, indent=4)

In [40]:
def gettimeGol(events):
    gols = list()

    for i in events:
        svg = i.find_element(By.TAG_NAME, 'svg')
        icon = svg.get_attribute("class")
        if 'soccer' in icon:
            timeGol = i.find_element(By.CLASS_NAME, 'smv__timeBox').text
            #timeGol = int(timeGol.replace("'", ""))
            gols.append(timeGol)

    return gols

In [ ]:
def getStats(driver):
    
    section =  WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="detail"]/div[9]'))
    )
    section = section.text
    
    stats = section.split("\n")
    statsHome = list(range(0,len(stats), 3))
    statsAway = list(range(2, len(stats), 3))
    categoria = list(range(1, len(stats), 3))

    estatisitcas = list()
    for i in range(len(categoria)):
        home = statsHome[i]
        away = statsAway[i]
        c = categoria[i]
        d = {'categoria': stats[c], 'home': stats[home], 'away': stats[away] }
        estatisitcas.append(d)
    
    return estatisitcas

In [41]:
def getData(driver, ID):

    home = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[2]/div[3]/div[2]/a')
    away = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[4]/div[3]/div[1]/a')
    time = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[1]/div')
    placar = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[3]/div/div[1]').text
    placarHaf = driver.find_element(By.CLASS_NAME, 'smv__incidentsHeader')
    placarHaf = placarHaf.text.split("\n")[1] 
    # try:
    #     placarHaf = driver.find_element(By.XPATH, '//*[@id="detail"]/div[9]/div/div[1]/div[2]').text
    # except:
    #     placarHaf = driver.find_element(By.XPATH, '//*[@id="detail"]/div[10]/div/div[1]/div[2]').text


    odds = driver.find_element(By.CLASS_NAME, 'oddsRowContent').text
    odds = odds.split("\n")
    oddsHome = odds[1]
    oddsDraw = odds[3]
    oddsAway = odds[5]

    # homeGols = driver.find_elements(By.CLASS_NAME, 'smv__homeParticipant')
    # homeGols = gettimeGol(homeGols)

    # awayGols = driver.find_elements(By.CLASS_NAME, 'smv__awayParticipant')
    # awayGols = gettimeGol(awayGols)
    
    allgols = driver.find_elements(By.CLASS_NAME, 'smv__participantRow')
    allgols = gettimeGol(allgols)    
    

    d = {
        'time': time.text,
        'home': home.text,
        'away': away.text,
        'placar': placar.replace('\n', ''),
        "placarHaf": placarHaf.replace('\n', ''),
        # 'placarHafHome': placarHaf.split(" - ")[0],
        # 'placarHafAway': placarHaf.split(" - ")[1],
        'oddsHome': oddsHome,
        'oddsDraw': oddsDraw,
        'oddsAway': oddsAway,
        # 'homeGols': homeGols,
        # "awayGols": awayGols,
        "allgols": allgols   
    }
    return d

In [52]:
for ID in IDS[71:]:
    index = IDS.index(ID)
    print('Extraindo: ', index)
    url = f"https://www.flashscore.com.br/jogo/{ID}"
    driver.get(url)
    time.sleep(3)
    data = getData(driver, ID)

    with open(f"data/diario/02_12/events/{index}.json","w", encoding='utf-8') as jsonfile:
            json.dump(data, jsonfile, ensure_ascii=False, indent=4)


Extraindo:  71
Extraindo:  72
Extraindo:  73
Extraindo:  74
Extraindo:  75
Extraindo:  76
Extraindo:  77
Extraindo:  78
Extraindo:  79
Extraindo:  80
Extraindo:  81
Extraindo:  82
Extraindo:  83


In [21]:
x = driver.find_element(By.CLASS_NAME, 'smv__incidentsHeader')

In [24]:
x.text.split("\n")[1]


'0 - 0'

In [13]:
data

{'time': '03.12.2022 11:00',
 'home': 'Países Baixos',
 'away': 'EUA',
 'placar': '3-1',
 'placarHaf': '2 - 0',
 'oddsHome': '1.90',
 'oddsDraw': '3.30',
 'oddsAway': '4.33',
 'allgols': ["10'", "45+1'", "76'", "81'"]}

In [ ]:
with open(f"out/{index}.json","w", encoding='utf-8') as jsonfile:
            json.dump(data, jsonfile, ensure_ascii=False, indent=4)

In [ ]:
def gettimeGol(events):
    gols = list()

    for i in events:
        svg = i.find_element(By.TAG_NAME, 'svg')
        icon = svg.get_attribute("class")
        if 'soccer' in icon:
            timeGol = i.find_element(By.CLASS_NAME, 'smv__timeBox').text
            #timeGol = int(timeGol.replace("'", ""))
            gols.append(timeGol)

    return gols

In [ ]:
def getData(driver):

    home = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[2]/div[3]/div[2]/a')
    away = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[4]/div[3]/div[1]/a')
    time = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[1]/div')
    placar = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[3]/div/div[1]').text

    homeGols = driver.find_elements(By.CLASS_NAME, 'smv__homeParticipant')
    homeGols = gettimeGol(homeGols)

    awayGols = driver.find_elements(By.CLASS_NAME, 'smv__awayParticipant')
    awayGols = gettimeGol(awayGols)
    
    allgols = driver.find_elements(By.CLASS_NAME, 'smv__participantRow')
    allgols = gettimeGol(allgols)
    
    

    d = {
        'time': time.text,
        'home': home.text,
        'away': away.text,
        'placar': placar.replace('\n', ''),
        'homeGols': homeGols,
        "awayGols": awayGols,
        "allgols": allgols       
    }
    return d
